In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from tqdm import tqdm
from Base.DataIO import DataIO

#----Recommenders----
from SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from SLIM.SlimElasticNet import SLIMElasticNetRecommender
from cf.item_cf3 import ItemBasedCollaborativeFiltering
from cf.user_cf2 import UserBasedCollaborativeFiltering
from MF.ALS import AlternatingLeastSquare
from cbf.cbf import ContentBasedFiltering
from SlimBPR.SlimBPRRec import SlimBPRRec
from SlimBPR.SlimBPR import SlimBPR
#---------------

#----Model Load & Save----
#from Data_manager.DataReader import DataReader
#from Data_manager.DataSplitter import DataSplitter
from Data_manager.Dataset import Dataset
#-------------------------

from sklearn.model_selection import train_test_split

**Dataset loading with pandas**

The function read_csv from pandas provides a wonderful and fast interface to load tabular data like this. For better results and performance we provide the separator ::, the column names ["user_id", "item_id", "ratings", "timestamp"], and the types of each attribute in the dtype parameter.

In [2]:
def load_data():
  return pd.read_csv("./data_train.csv")

In [3]:
ratings=load_data()
d ={'user_id': ratings['row'],'item_id':ratings['col'],'ratings':ratings['data']}
ratings=pd.DataFrame(data=d)

In [4]:
ratings.dtypes

user_id      int64
item_id      int64
ratings    float64
dtype: object

In [5]:
userList=list(d['user_id'])
itemList=list(d['item_id'])
ratingList=list(d['ratings'])

In [6]:
URM = sp.coo_matrix((ratingList,(userList,itemList)))
URM = URM.tocsr()

In [7]:
URM

<7947x25975 sparse matrix of type '<class 'numpy.float64'>'
	with 113268 stored elements in Compressed Sparse Row format>

In [8]:
def load_data_ICM():
  return pd.read_csv("./data_ICM_title_abstract.csv")

In [9]:
features=load_data_ICM()
d ={'item_id': features['row'],'feature_id':features['col'],'value':features['data']}
features=pd.DataFrame(data=d)
itemList=list(d['item_id'])

In [10]:
featureList=list(d['feature_id'])

In [11]:
valueList=list(d['value'])
ICM = sp.coo_matrix((valueList,(itemList,featureList)))
ICM = ICM.tocsr()

In [12]:
ICM

<25975x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 490691 stored elements in Compressed Sparse Row format>

In [13]:
num_users=URM.shape[0]

In [14]:
num_items=URM.shape[1]

In [15]:
num_users,num_items

(7947, 25975)

**Dataset splitting into train,validation and test**

This is the last part before creating the recommender. However, this step is super important, as it is the base for the training, parameters optimization, and evaluation of the recommender(s).

In here we read the ratings (which we loaded and preprocessed before) and create the train, validation, and test User-Rating Matrices (URM). It's important that these are disjoint to avoid information leakage from the train into the validation/test set, in our case, we are safe to use the train_test_split function from scikit-learn as the dataset only contains one datapoint for every (user,item) pair. On another topic, we first create the test set and then we create the validation by splitting again the train set.

train_test_split takes an array (or several arrays) and divides it into train and test according to a given size (in our case testing_percentage and validation_percentage, which need to be a float between 0 and 1).

After we have our different splits, we create the sparse URMs by using the csr_matrix function from scipy.




In [16]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Notebooks_utils.data_splitter import train_test_holdout

urm_train_validation, urm_test = train_test_holdout(URM, train_perc = 0.85)
urm_train, urm_validation = train_test_holdout(urm_train_validation, train_perc = 0.85)

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

In [17]:
dataset = Dataset("prova", urm_train_validation, urm_test, ICM) 

In [18]:
dataset.save_data("salvataggi")

  (0, 10080)	1.0
  (1, 2665)	1.0
  (1, 17068)	1.0
  (1, 17723)	1.0
  (1, 18131)	1.0
  (1, 20146)	1.0
  (2, 21181)	1.0
  (3, 18736)	1.0
  (3, 23037)	1.0
  (4, 477)	1.0
  (4, 6927)	1.0
  (4, 10204)	1.0
  (4, 13707)	1.0
  (4, 18999)	1.0
  (5, 814)	1.0
  (5, 2754)	1.0
  (5, 3907)	1.0
  (5, 4481)	1.0
  (5, 5581)	1.0
  (5, 6549)	1.0
  (5, 7583)	1.0
  (5, 8849)	1.0
  (5, 9014)	1.0
  (5, 9658)	1.0
  (5, 12914)	1.0
  :	:
  (7944, 22869)	1.0
  (7944, 23519)	1.0
  (7944, 23721)	1.0
  (7944, 23986)	1.0
  (7944, 24022)	1.0
  (7944, 24075)	1.0
  (7944, 24484)	1.0
  (7944, 24806)	1.0
  (7944, 24882)	1.0
  (7944, 24912)	1.0
  (7944, 24990)	1.0
  (7944, 25068)	1.0
  (7944, 25119)	1.0
  (7944, 25314)	1.0
  (7944, 25707)	1.0
  (7944, 25713)	1.0
  (7944, 25746)	1.0
  (7944, 25812)	1.0
  (7944, 25882)	1.0
  (7945, 817)	1.0
  (7945, 2476)	1.0
  (7945, 12319)	1.0
  (7945, 21384)	1.0
  (7946, 8699)	1.0
  (7946, 19178)	1.0
  (0, 19467)	1.0
  (1, 7494)	1.0
  (2, 19337)	1.0
  (4, 19838)	1.0
  (4, 19851)	1.0
  (7

AttributeError: items not found

In [ ]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["knn"] = Integer(5, 1000)
hyperparameters_range_dictionary["shrink"] = Integer(0, 1000)
hyperparameters_range_dictionary["similarity"] = Categorical(["cosine"])

In [ ]:
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = UserBasedCollaborativeFiltering

parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)

In [ ]:
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [ ]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [ ]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 5
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"

In [ ]:
parameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize,
                      )

In [ ]:
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
best_parameters = search_metadata["hyperparameters_best"]
best_parameters

**Evaluation Metrics**

In [ ]:
def recall(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant) / relevant_items.shape[0]
    
    return recall_score
    
    
def precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant) / recommendations.shape[0]

    return precision_score

def mean_average_precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(precision_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

**Evaluation Procedure**

The evaluation procedure returns the averaged accuracy scores (in terms of precision, recall and MAP) for all users (that have at least 1 rating in the test set). It also calculates the number of evaluated and skipped users. It receives a recommender instance, and the train and test URMs.

In [ ]:
def evaluator(recommender: object, urm_train: sp.csr_matrix, urm_test: sp.csr_matrix):
    recommendation_length = 10
    accum_precision = 0
    accum_recall = 0
    accum_map = 0
    
    num_users = urm_train.shape[0]
    
    num_users_evaluated = 0
    num_users_skipped = 0
    
    for user_id in range(num_users):
        user_profile_start = urm_test.indptr[user_id]
        user_profile_end = urm_test.indptr[user_id+1]

        relevant_items = urm_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
            
#         recommendations = recommender.recommend(user_id_array=user_id,
#                                                cutoff=recommendation_length,
#                                                remove_seen_flag=True
#                                                )

        expected_ratings = recommender.get_expected_ratings(user_id)
        recommended_items = np.flip(np.argsort(expected_ratings), 0)

        unseen_items_mask = np.in1d(recommended_items,urm_train[user_id].indices,
                                        assume_unique=True, invert=True)

        recommendations = recommended_items[unseen_items_mask]


        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1

    
    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)
    
    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped

In [ ]:
#Params for the recommenders
cbf_param = {
    "knn": 140,
    "shrink": 0
}

user_cf_param = {
    "knn": 98,
    "shrink": 364
}

item_cf_param = {
    "knn": 1000,
    "shrink": 562
}

slim_bpr_param = {
    "learning_rate" : 0.05,
    "epochs": 10,
    "nnz" : 1,
    "knn": 200
}

als_param = {
    "n_factors": 300,
    "regularization": 0.15,
    "iterations": 30
}


In [ ]:
userCF = UserBasedCollaborativeFiltering(knn=user_cf_param["knn"], shrink=user_cf_param["shrink"])

In [ ]:
userCF.fit(urm_train)

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(userCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
itemCF = ItemBasedCollaborativeFiltering(urm_train)

In [ ]:
itemCF.fit(knn=item_cf_param["knn"], shrink=item_cf_param["shrink"], similarity="cosine")

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
SlimBpr=SlimBPRRec(learning_rate=slim_bpr_param["learning_rate"], epochs=slim_bpr_param["epochs"], nnz=slim_bpr_param["nnz"], knn=slim_bpr_param["knn"])

In [ ]:
SlimBpr.fit(urm_train)

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(SlimBpr, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
slim_elastic = SLIMElasticNetRecommender()

In [ ]:
slim_elastic.fit(urm_train)

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(slim_elastic, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
cbf = ContentBasedFiltering(knn=cbf_param["knn"],shrink=cbf_param["shrink"])

In [ ]:
cbf.fit(urm_train,ICM)

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(cbf, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
ALS = AlternatingLeastSquare(n_factors=als_param["n_factors"], regularization=als_param["regularization"],iterations=als_param["iterations"])

In [ ]:
ALS.fit(urm_train)

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(ALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

**Hybrid**

In [ ]:
class HybridRecommender(object):
    def __init__(self, URM, ICM):
        
        self.urm = URM
        
        self.ICM = ICM

#         self.userCF = UserBasedCollaborativeFiltering(URM.copy())

        self.itemCF = ItemBasedCollaborativeFiltering(URM.copy())
        
#         self.cbf = ContentBasedFiltering(URM.copy(), ICM.copy())
        
#         self.slim_random = SLIM_BPR_Python(URM.copy())
        
#         self.slim_elastic = SLIMElasticNetRecommender()
        
#         self.ALS = AlternatingLeastSquare(URM.copy())
    

    def fit(self, w, user_cf_param, item_cf_param, cbf_param, slim_param, als_param,):
        self.URM = URM

        ### SUB-FITTING ###
        print("Fitting user cf...")
#         self.userCF.fit(knn=user_cf_param["knn"], shrink=user_cf_param["shrink"])

        print("Fitting item cf...")
        self.itemCF.fit(knn=item_cf_param["knn"], shrink=item_cf_param["shrink"], similarity="cosine")
        
        print("Fitting cbf...")
#         self.cbf.fit(knn=cbf_param["knn"],shrink=cbf_param["shrink"])
        
        print("Fitting slim bpr...")
#         self.slim_random.fit(topK=slim_param["topK"],epochs=slim_param["epochs"])
        
        print("Fitting slim elastic...")
#         self.slim_elastic.fit(URM.copy())
        
        print("Fitting ALS...")
#         self.ALS.fit(n_factors=als_param["n_factors"], regularization=als_param["regularization"],iterations=als_param["iterations"])


    def recommend(self,user_id,urm_train: sp.csr_matrix,at=10):
     
        expected_ratings = self.get_expected_ratings(user_id)
        
        recommended_items = np.flip(np.argsort(expected_ratings), 0)

        # REMOVING SEEN
        unseen_items_mask = np.in1d(recommended_items,urm_train[user_id].indices,
                                    assume_unique=True, invert=True)
        recommended_items = recommended_items[unseen_items_mask]

        return recommended_items[0:at]
    
    def get_expected_ratings(self, user_id) :
#         self.userCF_ratings = self.userCF.get_expected_ratings(user_id)
        self.itemCF_ratings = self.itemCF.get_expected_ratings(user_id)
#         self.cbf_ratings = self.cbf.get_expected_ratings(user_id)
#         self.slim_ratings = self.slim_random.get_expected_ratings(user_id)
#         self.slim_elastic_ratings = self.slim_elastic.get_expected_ratings(user_id)
#         self.ALS_ratings = self.ALS.get_expected_ratings(user_id)

        self.hybrid_ratings = None 

#         self.hybrid_ratings = self.userCF_ratings * w["user_cf"]
        self.hybrid_ratings = self.itemCF_ratings * w["item_cf"]
#         self.hybrid_ratings += self.cbf_ratings * w_right["cbf"]
#         self.hybrid_ratings += self.slim_ratings * w["slim"]
#         self.hybrid_ratings += self.ALS_ratings * w["als"]
#         self.hybrid_ratings += self.slim_elastic_ratings * w["elastic"]

        return self.hybrid_ratings

w = {
    "user_cf": 0,
    "item_cf": 1,
    "cbf": 0,
    "icm_svd": 0,
    "als": 0,
    "slim": 0,
    "elastic": 0
}


In [ ]:
recommender = HybridRecommender(urm_train, ICM)

In [ ]:
recommender.fit(w, user_cf_param=user_cf_param,item_cf_param=item_cf_param,cbf_param=cbf_param,slim_param=slim_bpr_param,als_param=als_param)

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommender, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

**Submission to competition**

This step serves as a similar step that you will perform when preparing a submission to the competition. Specially after you have chosen and trained your recommender.

For this step the best suggestion is to select the most-performing configuration obtained in the hyperparameter tuning step and to train the recommender using both the train and validation set. Remember that in the competition you do not have access to the test set.

Another consideration is that, due to easier and faster calculations, we replaced the user/item identifiers with new ones in the preprocessing step. For the competition, you are required to generate recommendations using the dataset's original identifiers. Due to this, this step also reverts back the newer identifiers with the ones originally found in the dataset.

Last, this step creates a function that writes the recommendations for each user in the same file in a tabular format following this format:

csv
<user_id>,<item_id_1> <item_id_2> <item_id_3> <item_id_4> <item_id_5> <item_id_6> <item_id_7> <item_id_8> <item_id_9> <item_id_10>
Always verify the competitions' submission file model as it might vary from the one we presented here.

In [ ]:
def load_goodguys():
  return pd.read_csv("./data_target_users_test.csv")
goodguys=load_goodguys()

In [ ]:
goodguys

In [ ]:
users_to_recommend = np.random.choice(goodguys.user_id,size=goodguys.size, replace=False)
users_to_recommend

In [ ]:
def prepare_submission(ratings: pd.DataFrame, users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object):
    
    recommendation_length = 10
    submission = []
    
    for user_id in users_to_recommend :

        recommendations = recommender.recommend(user_id, urm_train, recommendation_length)

        
        submission.append((user_id, [item_id for item_id in recommendations]))
   
    return submission

In [ ]:
submission = prepare_submission(ratings, users_to_recommend, urm_train_validation, recommender)

In [ ]:
submission

In [ ]:
import os
from datetime import datetime

csv_fname = './submission'
csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

def write_submission(submissions):
    with open(csv_fname, "w") as f:
        f.write(f"user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")


In [ ]:
write_submission(submission)